In [25]:
# @title Install and Setup - Upload Files
!pip install onnx onnxruntime opencv-python insightface gdown tqdm

import datetime
import numpy as np
import os
import os.path as osp
import glob
import cv2
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import matplotlib.pyplot as plt
import gdown
from PIL import Image
from google.colab import files
from IPython.display import clear_output
from tqdm import tqdm

def upload_files():
    uploaded = files.upload()
    return list(uploaded.keys())

#/root/.insightface/models/buffalo_l.zip
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))
# Import the library
import gdown

# Google Drive file ID
file_id = '1krOLgjW2tAPaqV-Bw4YALz0xT5zlb5HF'
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, 'inswapper_128.onnx', quiet=False)

swapper = insightface.model_zoo.get_model('inswapper_128.onnx',download=False,download_zip=False)

clear_output()

face_image = upload_files()
image_files = upload_files()

clear_output()

In [49]:
# @title Run FaceSwap
def faceswap(face_image,image_files,path=".",directory="."):
  img1 = cv2.imread(face_image)
  img_lst = [cv2.imread(path + "/" + img) for img in image_files]
  base_face = app.get(img1)[0]
  face_lst =[app.get(face)[0] for face in img_lst]
  out_img = [swapper.get(swap.copy(), face_lst[i], base_face, paste_back=True) for i,swap in enumerate(tqdm(img_lst, desc="Please Wait"))]
  swapped_img1_rgb = [cv2.cvtColor(swapped, cv2.COLOR_BGR2RGB) for swapped in out_img]

  for i,swapped2 in enumerate(swapped_img1_rgb):
    Image.fromarray(np.uint8(swapped2)).save(f"{directory}/{i:06}.jpg")

def faceswapdir(face_image,path="."):
  img1 = cv2.imread(face_image)
  img_lst = [cv2.imread(f"{path}/{img}") for img in os.listdir(path)]
  base_face = app.get(img1)[0]
  face_lst =[app.get(face)[0] for face in img_lst]
  out_img = [swapper.get(swap.copy(), face_lst[i], base_face, paste_back=True) for i,swap in enumerate(tqdm(img_lst, desc="Please Wait"))]
  swapped_img1_rgb = [cv2.cvtColor(swapped, cv2.COLOR_BGR2RGB) for swapped in out_img]
  !mkdir output
  for i,swapped2 in enumerate(swapped_img1_rgb):
    Image.fromarray(np.uint8(swapped2)).save(f"output/{i:06}.jpg")


#faceswap(face_image[0],image_files,"/content/frame2")


In [ ]:
# @title Only face upload
face_image = upload_files()
print('insightface', insightface.__version__)
print('numpy', np.__version__)

# Check that InsightFace version is at least 0.7
assert float('.'.join(insightface.__version__.split('.')[:2])) >= float('0.7')

In [ ]:
# @title Only images upload
image_files = upload_files()


In [ ]:
face_image

In [ ]:
!ffmpeg -i sample.mov -vcodec h264 -acodec aac sample.mp4

In [ ]:
!ffmpeg -i sample.mp4 -q:a 0 -map a sample.mp3

In [ ]:
!ffmpeg -i sample.mp4 -q:v 2 frame/%06d.jpg

In [50]:
fil = os.listdir('frame')
fil.sort()
print(fil)
faceswapdir(face_image[0],"/content/frame")

['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg', '000011.jpg', '000012.jpg', '000013.jpg', '000014.jpg', '000015.jpg', '000016.jpg', '000017.jpg', '000018.jpg', '000019.jpg', '000020.jpg', '000021.jpg', '000022.jpg', '000023.jpg', '000024.jpg', '000025.jpg', '000026.jpg', '000027.jpg', '000028.jpg', '000029.jpg', '000030.jpg', '000031.jpg', '000032.jpg', '000033.jpg', '000034.jpg', '000035.jpg', '000036.jpg', '000037.jpg']


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
!ffmpeg -f image2 -framerate 30 -i frame/%06d.jpg -c:v libx264 out.mp4

In [ ]:
!ffmpeg -i video.mp4 -i audio.wav -c:v copy -c:a aac output.mp4

In [ ]:
!ffmpeg -copyts -ss 00:00:01 -i sample.mp4 -to 00:00:03 -map 0 -c copy out.mp4